In [1]:
import numpy as np

index = np.load("/home/jovyan/Sample_Based_Extension/UNSW/best_defense_indices.npy", allow_pickle=True)
index

array([['baseline', 0, 'Def1-11 Combined', 0.9923313825446824,
        0.9911800925969834, array([9, 3, 9, ..., 9, 9, 9])],
       ['BIM', 0.01, 'Def1-11 Combined', 0.9891551359556379,
        0.987484040917074, array([9, 3, 9, ..., 9, 9, 9])],
       ['BIM', 0.1, 'Def1-11 Combined', 0.9746251239350361,
        0.9705688736459472, array([1, 6, 1, ..., 1, 1, 1])],
       ['BIM', 0.2, 'Def1-11 Combined', 0.9594545980995165,
        0.9538946564599001, array([6, 6, 6, ..., 6, 6, 6])],
       ['BIM', 0.3, 'Def1-11 Combined', 0.919233840187416,
        0.9113183206844415, array([6, 6, 6, ..., 6, 6, 6])],
       ['FGSM', 0.01, 'Def1-11 Combined', 0.9889357819094331,
        0.9872271983425641, array([9, 3, 9, ..., 9, 9, 9])],
       ['FGSM', 0.1, 'Def1-11 Combined', 0.9680708250344385,
        0.9628361221532343, array([1, 6, 1, ..., 1, 1, 1])],
       ['FGSM', 0.2, 'Def1-11 Combined', 0.9461968395469023,
        0.9384213027985543, array([1, 6, 1, ..., 1, 1, 1])],
       ['FGSM', 0.3, 'Def1

In [2]:
attack_models = [
    ("BIM", [0.1]),
    ("FGSM", [0.1]),
    ("PGD", [0.1]),
    ("DF", [0.1]),
    ("AutoPGD", [0.1]),
    ("ZOO", [0.1]),
    ("CaFA", [0.1]),
    ("SINIFGSM", [0.1]),
    ("VNIFGSM", [0.1]),
]

all_adv_examples = []

for attack_name, epsilons in attack_models:
    for epsilon in epsilons:
        filename = f'/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
        x_test_adv = np.load(filename)
        all_adv_examples.append(x_test_adv)

x_test_adv_0_1 = np.concatenate(all_adv_examples, axis=0)


In [3]:
y_labels = []

for row in index:
    attack_name, eps = row[0], row[1]
    if eps == 0.1:
        y_array = row[-1]
        y_labels.append(y_array)

y_label_0_1 = np.concatenate(y_labels, axis=0)

In [4]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
label_encoder.fit(y_label_0_1)

y_label_0_1 = label_encoder.transform(y_label_0_1)

for index, class_label in enumerate(label_encoder.classes_):
    print(f"defense '{class_label}' encode {index}")

defense '1' encode 0
defense '2' encode 1
defense '3' encode 2
defense '4' encode 3
defense '5' encode 4
defense '6' encode 5
defense '7' encode 6
defense '8' encode 7
defense '9' encode 8
defense '10' encode 9
defense '11' encode 10


In [1]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.cluster import MiniBatchKMeans
import numpy as np
from scipy.sparse import vstack

def stratified_random_sample(X, y, fraction=None, count=None, random_state=42):
    assert (fraction is not None) ^ (count is not None), "Specify either fraction or count"

    rng = np.random.RandomState(seed=random_state)
    X_out, y_out, indices_out = [], [], []

    for label in np.unique(y):
        X_c = X[y == label]
        y_c = y[y == label]
        indices_c = np.where(y == label)[0]

        n_total = len(X_c)
        n_sample = int(n_total * fraction) if fraction is not None else min(count, n_total)
        # Ensure at least one sample per class
        n_sample = max(1, n_sample)

        idx = rng.choice(n_total, size=n_sample, replace=False)

        X_out.append(X_c[idx])
        y_out.append(y_c[idx])
        indices_out.append(indices_c[idx])

    return np.concatenate(X_out, axis=0), np.concatenate(y_out, axis=0), np.concatenate(indices_out, axis=0)

def batch_mode_active_learning(X_pool, y_pool, model, fraction=0.2, random_state=42, max_clusters_per_iter=1000, max_top_k=10000, batch_size=10000):
    selected_indices = []
    rng = np.random.RandomState(seed=random_state)
    n_total = len(X_pool)
    n_select = max(1, int(n_total * fraction))  # 目标选择样本数
    classes = np.unique(y_pool)

    # Convert sparse to dense if necessary (handle in batches if needed)
    is_sparse = hasattr(X_pool, 'toarray')

    # Step 1: Ensure at least one sample per class directly from X_pool
    for c in classes:
        idx_c = np.where(y_pool == c)[0]
        if len(idx_c) == 0:
            continue
        # Randomly select one sample for this class
        selected_idx = rng.choice(idx_c, size=1)[0]
        selected_indices.append(selected_idx)

    # Step 2: Compute uncertainties for X_pool in batches
    uncertainties = np.zeros(n_total)
    for start in range(0, n_total, batch_size):
        end = min(start + batch_size, n_total)
        X_batch = X_pool[start:end].toarray() if is_sparse else X_pool[start:end]
        probs = model.predict_proba(X_batch)
        uncertainties[start:end] = 1 - np.max(probs, axis=1)

    # Step 3: Iterative clustering to select remaining samples
    n_remaining = n_select - len(selected_indices)
    current_indices = np.setdiff1d(np.arange(n_total), selected_indices)  # 剩余样本索引

    while n_remaining > 0 and len(current_indices) > 0:
        # Determine clusters for this iteration
        n_clusters = min(max_clusters_per_iter, n_remaining, len(current_indices))
        if n_clusters == 0:
            break

        # Select top-k uncertain instances
        top_k = min(n_clusters * 5, len(current_indices) - 1, max_top_k)
        if top_k <= 0:
            break

        current_uncertainties = uncertainties[current_indices]
        top_indices = np.argpartition(-current_uncertainties, top_k)[:top_k]
        top_global_indices = current_indices[top_indices]

        # Prepare data for clustering
        X_top = X_pool[top_global_indices].toarray() if is_sparse else X_pool[top_global_indices]

        # Adjust n_clusters to be at most the number of samples
        n_clusters = min(n_clusters, len(top_indices))
        if n_clusters == 0:
            break

        # Cluster with MiniBatchKMeans
        kmeans = MiniBatchKMeans(n_clusters=n_clusters, random_state=random_state, batch_size=1000, max_iter=100)
        kmeans.fit(X_top)

        # Vectorized distance calculation
        distances = np.linalg.norm(X_top[:, np.newaxis] - kmeans.cluster_centers_, axis=2)
        cluster_closest = np.argmin(distances, axis=0)
        selected_indices.extend(top_global_indices[cluster_closest])

        # Update remaining samples and uncertainties
        current_indices = np.setdiff1d(current_indices, top_global_indices[cluster_closest])
        n_remaining -= n_clusters

    # Step 4: Finalize outputs
    selected_indices = np.array(selected_indices[:n_select])  # Trim if oversampled
    X_selected = X_pool[selected_indices]
    y_selected = y_pool[selected_indices]
    remaining_idx = np.setdiff1d(np.arange(n_total), selected_indices)
    X_rest = X_pool[remaining_idx]
    y_rest = y_pool[remaining_idx]

    # Step 5: Verify each class appears at least once
    selected_class_counts = np.bincount(y_selected, minlength=len(classes))
    missing_classes = classes[selected_class_counts == 0]
    if len(missing_classes) > 0:
        print(f"Warning: Missing classes {missing_classes}. Adding one sample per missing class.")
        for c in missing_classes:
            idx_c = np.where(y_pool == c)[0]
            if len(idx_c) > 0:
                selected_idx = rng.choice(idx_c, size=1)[0]
                selected_indices = np.append(selected_indices, selected_idx)
                n_select += 1  # Adjust n_select to account for extra samples
        # Update outputs after adding missing classes
        selected_indices = selected_indices[:n_select]
        X_selected = X_pool[selected_indices]
        y_selected = y_pool[selected_indices]
        remaining_idx = np.setdiff1d(np.arange(n_total), selected_indices)
        X_rest = X_pool[remaining_idx]
        y_rest = y_pool[remaining_idx]

    # Print selected samples' label distribution
    print(f"Selected samples (fraction={fraction}): {len(y_selected)}")
    print(f"Label distribution: {np.bincount(y_selected, minlength=len(classes))}")

    return X_rest, X_selected, y_rest, y_selected

X_pool = x_test_adv_0_1 
y_pool = y_label_0_1
X_train_init, y_train_init, _ = stratified_random_sample(X_pool, y_pool, fraction=0.1, random_state=42)

clf = RandomForestClassifier(random_state=42)
clf.fit(X_train_init, y_train_init)
# Perform batch mode active learning with fractions 50%, 10%, 1%
_, X_10, _, y_10 = batch_mode_active_learning(
    x_test_adv_0_1, y_label_0_1, clf, fraction=0.1, random_state=42
)



NameError: name 'x_test_adv_0_1' is not defined

In [23]:
X_10.shape


(205147, 56)

In [24]:
# from xgboost import XGBClassifier

# print('xgb')
# xgb10 = XGBClassifier()
# xgb10.fit(X_10, y_10)

print("RF")
from sklearn.ensemble import RandomForestClassifier
rf10 = RandomForestClassifier()
rf10.fit(X_10, y_10)

# print("DT")
# from sklearn.tree import DecisionTreeClassifier
# dt10 = DecisionTreeClassifier()
# dt10.fit(X_10, y_10)

xgb
RF
DT


DecisionTreeClassifier()

In [ ]:
import time
import numpy as np

y_test = np.load('/home/jovyan/Sample_Based_Extension/UNSW/y_test.npy')

attack_models = [
    ("baseline", [0]),
    ("BIM", [0.01, 0.2, 0.3]),
    ("FGSM", [0.01, 0.2, 0.3]),
    ("PGD", [0.01, 0.2, 0.3]),
    ("DF", [0.01, 0.2, 0.3]),
    ("AutoPGD", [0.01, 0.2, 0.3]),
    ("ZOO", [0.01, 0.2, 0.3]),
    ("CaFA", [0.01, 0.2, 0.3]),
    ("SINIFGSM", [0.01, 0.2, 0.3]),
    ("VNIFGSM", [0.01, 0.2, 0.3]),
]

In [25]:


# base_path_template = "/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_{attack}_eps_{eps}.npy"

# from sklearn.model_selection import train_test_split

# for attack_name, epsilons in attack_models:
#     start_time = time.time()
#     all_adv_examples = []
#     all_y = []
    
#     if attack_name != "baseline":
#         for epsilon in epsilons:
#             filename = f'/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
#             x_test_adv = np.load(filename)
#             all_adv_examples.append(x_test_adv)
#             all_y.append(y_test)
    
#         X = np.concatenate(all_adv_examples, axis=0)
#         y = np.concatenate(all_y, axis=0)
#     else:
#         X = np.load('/home/jovyan/Sample_Based_Extension/UNSW/x_test.npy')
#         y = np.load('/home/jovyan/Sample_Based_Extension/UNSW/y_test.npy')

#     y_pred = xgb10.predict(X)
#     print(y_pred.shape)
#     unique_classes, count = np.unique(y_pred, return_counts=True)
#     print(unique_classes, count)
#     for class_num in unique_classes:
#         indices = np.where(y_pred == class_num)[0]
#         X_class = X[indices]
#         y_class = y[indices]
#         print(X_class.shape, y_class.shape)
    
#         x_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/XGB_ActiveLearning_BatchMode/UNSW_Input10/x_test_adv_{attack_name}_Def{class_num+1}.npy"
#         y_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/XGB_ActiveLearning_BatchMode/UNSW_Input10/y_test_adv_{attack_name}_Def{class_num+1}.npy"
        
#         np.save(x_filename, X_class)
#         np.save(y_filename, y_class)
    
#         print(f"Save {class_num} to {attack_name}_Def{class_num+1}.npy")

#     end_time = time.time()
#     result = end_time - start_time
#     print(f"Execution Time: {result:.6f} seconds")
    

(113971,)
[ 0  1  2  3  4  5  6  7  8  9 10] [43730   178  2138   848    82  4261     1   201 62254   213    65]
(43730, 56) (43730,)
Save 0 to baseline_Def1.npy
(178, 56) (178,)
Save 1 to baseline_Def2.npy
(2138, 56) (2138,)
Save 2 to baseline_Def3.npy
(848, 56) (848,)
Save 3 to baseline_Def4.npy
(82, 56) (82,)
Save 4 to baseline_Def5.npy
(4261, 56) (4261,)
Save 5 to baseline_Def6.npy
(1, 56) (1,)
Save 6 to baseline_Def7.npy
(201, 56) (201,)
Save 7 to baseline_Def8.npy
(62254, 56) (62254,)
Save 8 to baseline_Def9.npy
(213, 56) (213,)
Save 9 to baseline_Def10.npy
(65, 56) (65,)
Save 10 to baseline_Def11.npy
Execution Time: 0.365381 seconds
(341913,)
[ 0  1  2  3  4  5  6  7  8  9 10] [178139    471   6176   7491    359  49846     15    189  99070    142
     15]
(178139, 56) (178139,)
Save 0 to BIM_Def1.npy
(471, 56) (471,)
Save 1 to BIM_Def2.npy
(6176, 56) (6176,)
Save 2 to BIM_Def3.npy
(7491, 56) (7491,)
Save 3 to BIM_Def4.npy
(359, 56) (359,)
Save 4 to BIM_Def5.npy
(49846, 56) (4984

In [26]:
from sklearn.model_selection import train_test_split

for attack_name, epsilons in attack_models:
    start_time = time.time()
    all_adv_examples = []
    all_y = []
    
    if attack_name != "baseline":
        for epsilon in epsilons:
            filename = f'/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
            x_test_adv = np.load(filename)
            all_adv_examples.append(x_test_adv)
            all_y.append(y_test)
    
        X = np.concatenate(all_adv_examples, axis=0)
        y = np.concatenate(all_y, axis=0)
    else:
        X = np.load('/home/jovyan/Sample_Based_Extension/UNSW/x_test.npy')
        y = np.load('/home/jovyan/Sample_Based_Extension/UNSW/y_test.npy')

    y_pred = rf10.predict(X)
    print(y_pred.shape)
    unique_classes, count = np.unique(y_pred, return_counts=True)
    print(unique_classes, count)
    for class_num in unique_classes:
        indices = np.where(y_pred == class_num)[0]
        X_class = X[indices]
        y_class = y[indices]
        print(X_class.shape, y_class.shape)
    
        x_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/RF_ActiveLearning_BatchMode/UNSW_Input10/x_test_adv_{attack_name}_Def{class_num+1}.npy"
        y_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/RF_ActiveLearning_BatchMode/UNSW_Input10/y_test_adv_{attack_name}_Def{class_num+1}.npy"
        
        np.save(x_filename, X_class)
        np.save(y_filename, y_class)
    
        print(f"Save {class_num} to {attack_name}_Def{class_num+1}.npy")

    end_time = time.time()
    result = end_time - start_time
    print(f"Execution Time: {result:.6f} seconds")
    

(113971,)
[ 0  1  2  3  4  5  6  7  8  9 10] [39968   308  1697   868    91  4270     1   208 66235   274    51]
(39968, 56) (39968,)
Save 0 to baseline_Def1.npy
(308, 56) (308,)
Save 1 to baseline_Def2.npy
(1697, 56) (1697,)
Save 2 to baseline_Def3.npy
(868, 56) (868,)
Save 3 to baseline_Def4.npy
(91, 56) (91,)
Save 4 to baseline_Def5.npy
(4270, 56) (4270,)
Save 5 to baseline_Def6.npy
(1, 56) (1,)
Save 6 to baseline_Def7.npy
(208, 56) (208,)
Save 7 to baseline_Def8.npy
(66235, 56) (66235,)
Save 8 to baseline_Def9.npy
(274, 56) (274,)
Save 9 to baseline_Def10.npy
(51, 56) (51,)
Save 10 to baseline_Def11.npy
Execution Time: 2.305631 seconds
(341913,)
[ 0  1  2  3  4  5  7  8  9 10] [135913     22   2037  15024    157 105931     42  82712      5     70]
(135913, 56) (135913,)
Save 0 to BIM_Def1.npy
(22, 56) (22,)
Save 1 to BIM_Def2.npy
(2037, 56) (2037,)
Save 2 to BIM_Def3.npy
(15024, 56) (15024,)
Save 3 to BIM_Def4.npy
(157, 56) (157,)
Save 4 to BIM_Def5.npy
(105931, 56) (105931,)
Save 

In [27]:
# from sklearn.model_selection import train_test_split

# for attack_name, epsilons in attack_models:
#     start_time = time.time()
#     all_adv_examples = []
#     all_y = []
    
#     if attack_name != "baseline":
#         for epsilon in epsilons:
#             filename = f'/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
#             x_test_adv = np.load(filename)
#             all_adv_examples.append(x_test_adv)
#             all_y.append(y_test)
    
#         X = np.concatenate(all_adv_examples, axis=0)
#         y = np.concatenate(all_y, axis=0)
#     else:
#         X = np.load('/home/jovyan/Sample_Based_Extension/UNSW/x_test.npy')
#         y = np.load('/home/jovyan/Sample_Based_Extension/UNSW/y_test.npy')

#     y_pred = dt10.predict(X)
#     print(y_pred.shape)
#     unique_classes, count = np.unique(y_pred, return_counts=True)
#     print(unique_classes, count)
#     for class_num in unique_classes:
#         indices = np.where(y_pred == class_num)[0]
#         X_class = X[indices]
#         y_class = y[indices]
#         print(X_class.shape, y_class.shape)
    
#         x_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/DT_ActiveLearning_BatchMode/UNSW_Input10/x_test_adv_{attack_name}_Def{class_num+1}.npy"
#         y_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/DT_ActiveLearning_BatchMode/UNSW_Input10/y_test_adv_{attack_name}_Def{class_num+1}.npy"
        
#         np.save(x_filename, X_class)
#         np.save(y_filename, y_class)
    
#         print(f"Save {class_num} to {attack_name}_Def{class_num+1}.npy")

#     end_time = time.time()
#     result = end_time - start_time
#     print(f"Execution Time: {result:.6f} seconds")
    

(113971,)
[ 0  1  2  3  4  5  6  7  8  9 10] [56055   841  2951  3661   302 12361     1   323 36901   315   260]
(56055, 56) (56055,)
Save 0 to baseline_Def1.npy
(841, 56) (841,)
Save 1 to baseline_Def2.npy
(2951, 56) (2951,)
Save 2 to baseline_Def3.npy
(3661, 56) (3661,)
Save 3 to baseline_Def4.npy
(302, 56) (302,)
Save 4 to baseline_Def5.npy
(12361, 56) (12361,)
Save 5 to baseline_Def6.npy
(1, 56) (1,)
Save 6 to baseline_Def7.npy
(323, 56) (323,)
Save 7 to baseline_Def8.npy
(36901, 56) (36901,)
Save 8 to baseline_Def9.npy
(315, 56) (315,)
Save 9 to baseline_Def10.npy
(260, 56) (260,)
Save 10 to baseline_Def11.npy
Execution Time: 0.100096 seconds
(341913,)
[ 0  1  2  3  4  5  6  7  8  9 10] [125034   4919  18895  25670   5821  75063  21394   6884  54872   1565
   1796]
(125034, 56) (125034,)
Save 0 to BIM_Def1.npy
(4919, 56) (4919,)
Save 1 to BIM_Def2.npy
(18895, 56) (18895,)
Save 2 to BIM_Def3.npy
(25670, 56) (25670,)
Save 3 to BIM_Def4.npy
(5821, 56) (5821,)
Save 4 to BIM_Def5.npy
(

In [28]:
import numpy as np


attack_names = [
    "baseline", 
    "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]

all_attack_list = []

model_name_list = ["XGB", "RF", "DT"]

for m_name in model_name_list:
    for attack_name in attack_names:
        each = []
        num = 0
        each.append(f"{m_name}10")
        for i in range(1, 12):
            y_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/{m_name}_ActiveLearning_BatchMode/UNSW_Input10/x_test_adv_{attack_name}_Def{i}.npy"
            try:
                y = np.load(y_filename)
                num+=y.shape[0]
                each.append(y.shape[0])
            except FileNotFoundError:
                each.append(0)
        each.append(num)
        all_attack_list.append(each)


In [29]:
import pandas as pd
all_attack_list = pd.DataFrame(all_attack_list)
all_attack_list.to_csv("./Dynamite_Num_Attack_Input10_ActiveLearning_BatchMode.csv")

In [30]:
all_attack_list

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,XGB20,43730,178,2138,848,82,4261,1,201,62254,213,65,113971
1,XGB20,178139,471,6176,7491,359,49846,15,189,99070,142,15,341913
2,XGB20,161739,163,4029,6952,507,61567,11,13814,93062,48,21,341913
3,XGB20,178139,471,6176,7491,359,49846,15,189,99070,142,15,341913
4,XGB20,176979,850,5328,221,388,103314,14890,36772,3041,72,58,341913
5,XGB20,214465,706,7357,5899,371,67871,14,782,43818,443,187,341913
6,XGB20,130870,529,6647,3222,218,12173,3,603,186829,624,195,341913
7,XGB20,154367,120,597,3998,220,95330,1,135,78801,7262,1082,341913
8,XGB20,171036,490,3819,5757,352,63324,6,217,96763,136,13,341913
9,XGB20,178547,129,4390,6703,595,58361,2,68,93052,51,15,341913
